In [ ]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize 

In [ ]:
with open(' ', 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data)

### Removing missing values:
Eliminate the row where is not revenue value for any year 2020, 2019, 2018, 2017, 2016. 

In [ ]:
# Eliminate the row where is not revenue value for any year 2020, 2019, 2018, 2017, 2016. 
# Later we return to discard the missing values in the REVENUE variable (target)
df= df.dropna(subset=['revenue.2020','revenue.2019','revenue.2018','revenue.2017','revenue.2016',], how='all')
df.shape

### Removing extrem values

In [ ]:
# Removing extreme values 2020 
# COVID extreme values:
df = df[df.vat_number != 'BE0463648419']   #PFIZER INNOVATIVE SUPPLY POINT INTERNATIONAL = BE0463648419
df = df[df.vat_number != 'BE0478242365']   #PFIZER SERVICE COMPANY = BE0478242365
df = df[df.vat_number != 'BE0416375270']   #EXXONMOBIL PETROLEUM & CHEMICAL


In [ ]:
# Removing extreme values 2019
df = df[df.vat_number != 'BE0822358674']     #GMED HEALTHCARE
df = df[df.vat_number != 'BE0419457296']     #ONTEX
df = df[df.vat_number != 'BE0473416418']     #TELENET
df = df[df.vat_number != 'BE0405414072']     #UNILIN
df = df[df.vat_number != 'BE0438282424']     #TECH DATA
df = df[df.vat_number != 'BE0403196039']     #PHILIP MORRIS BENELUX
df = df[df.vat_number != 'BE0403768933']     #ESTEE LAUDER
df = df[df.vat_number != 'BE0888256714']     #ST. JUDE MEDICAL COORDINATION CENTER
df = df[df.vat_number != 'BE0467071826']     #BLACK & DECKER LIMITED
df = df[df.vat_number != 'BE0464298616']     #AVNET EUROPE

In [ ]:
df.shape

### Code for melteing the Data:

In [ ]:
df_revenue = pd.melt(df, id_vars=['nace_code','vat_number'], value_vars=['revenue.2020', 'revenue.2019','revenue.2018', 'revenue.2017', 'revenue.2016', 'revenue.2015'], var_name='year_revenue',value_name='revenue')
df_revenue['id'] = df_revenue.nace_code.str.cat(df_revenue.vat_number)
df_revenue['year']=df_revenue['year_revenue'].apply(lambda x: x[-4:])
df_revenue['id'] = df_revenue.id.str.cat(df_revenue.year)
df_revenue.drop(['nace_code','vat_number','year'], axis='columns', inplace=True)

df_ebitda = pd.melt(df, id_vars=['company_name','nace_code','vat_number','creation_date','company_category', 'province'], value_vars=['ebitda.2020','ebitda.2019', 'ebitda.2018', 'ebitda.2017','ebitda.2016', 'ebitda.2015'], var_name='year_ebitda',value_name='ebitda')
df_ebitda['id'] = df_ebitda.nace_code.str.cat(df_ebitda.vat_number)
df_ebitda['year']=df_ebitda['year_ebitda'].apply(lambda x: x[-4:])
df_ebitda['id'] = df_ebitda.id.str.cat(df_ebitda.year)

df_assets = pd.melt(df, id_vars=['nace_code','vat_number'], value_vars=['total_assets.2020','total_assets.2019', 'total_assets.2018', 'total_assets.2017','total_assets.2016', 'total_assets.2015'], var_name='year_assets',value_name='total_assets')
df_assets['id'] = df_assets.nace_code.str.cat(df_assets.vat_number)
df_assets['year']=df_assets['year_assets'].apply(lambda x: x[-4:])
df_assets['id'] = df_assets.id.str.cat(df_assets.year)
df_assets.drop(['nace_code','vat_number','year'], axis='columns', inplace=True)

df_staff_cost = pd.melt(df, id_vars=['nace_code','vat_number'], value_vars=['staff_costs.2020', 'staff_costs.2019', 'staff_costs.2018',
       'staff_costs.2016', 'staff_costs.2017', 'staff_costs.2015'], var_name='year_staff_cost',value_name='staff_cost')
df_staff_cost['id'] = df_staff_cost.nace_code.str.cat(df_staff_cost.vat_number)
df_staff_cost['year']=df_staff_cost['year_staff_cost'].apply(lambda x: x[-4:])
df_staff_cost['id'] = df_staff_cost.id.str.cat(df_staff_cost.year)
df_staff_cost.drop(['nace_code','vat_number','year'], axis='columns', inplace=True)


In [ ]:
df2 = pd.merge(pd.merge(df_ebitda,df_assets,on='id'), df_staff_cost, on='id')
df_melt = pd.merge(df2, df_revenue, on='id')
df_melt.drop(['year_ebitda','year_assets','year_staff_cost','year_revenue'], axis=1, inplace=True)


Function for coding 'company_category'

In [ ]:
def code_size(x):
    '''
    Function for coding the variable company_category
    It is part of the features
    '''
    if x =='Small':
        return 1
    if x =='Medium sized':
        return 2
    if x =='Large':
        return 3
    if x =='Very large':
        return 4

df_melt['size_code'] = df_melt['company_category'].apply(code_size)


In [ ]:
#Transform string to date
df_melt['DateFormat'] = pd.to_datetime(df_melt.creation_date, format="%Y/%m/%d")
#Extracting Year
df_melt['creation_year'] = df_melt['DateFormat'].dt.year


In [ ]:
# Checking missing values
df_melt.isnull().sum()

### Drop remainder missing values from the REVENUE variable (target)

In [ ]:
# Drop remainder missing values from the REVENUE variable (target)
df_melt= df_melt.dropna(subset=['revenue'], how='all')
df_melt.shape

In [ ]:
# we can check that missing values from other variables decreased:
df_melt.isnull().sum()

### Creating the DataFrame Reference for filling the missing values

#### DataFrame reference: Company size, province, the highest level of the Nace code and year.

In [ ]:
# Creating a new nace code, only the highest level 
df_melt['highLevel_NaceCode']=df_melt['nace_code'].apply(lambda x: x[0:2])

In [ ]:
# Grouping 'company_category','province','new_naceCode','year' by median 
grouped_median = df_melt.groupby(['company_category','province','highLevel_NaceCode','year']).agg({'ebitda': ['median']})
grouped_median

### Filling missing values base on its reference 
(company size, province, the highest level of the Nace code and year)

In [ ]:
# Filling Nan values for every selected Feature (with missing values):

#ebitda
median_ebitda = df_melt.groupby(['company_category','province','highLevel_NaceCode','year'])['ebitda'].transform('median')
df_melt.loc[df_melt['ebitda'].isnull(), 'ebitda'] = median_ebitda

#total_assets
median_totalAssets = df_melt.groupby(['company_category','province','highLevel_NaceCode','year'])['total_assets'].transform('median')
df_melt.loc[df_melt['total_assets'].isnull(), 'total_assets'] = median_totalAssets

#staff_cost
median_staffCost = df_melt.groupby(['company_category','province','highLevel_NaceCode','year'])['staff_cost'].transform('median')
df_melt.loc[df_melt['staff_cost'].isnull(), 'staff_cost'] = median_staffCost



Checking remainder missing values:

In [ ]:
df_melt.isnull().sum()

#### Reducing the DataFrame reference one level for filling remainder missing values

'Company size, province and the highest level of the Nace code'

In [ ]:
#for Ebitda, reducing the three (DataFrame reference) one level
median_ebitda2 = df_melt.groupby(['company_category','province','highLevel_NaceCode'])['ebitda'].transform('median')
df_melt.loc[df_melt['ebitda'].isnull(), 'ebitda'] = median_ebitda2

#staff_cost, reducing the three (DataFrame reference) one level
median_staff_Cost = df_melt.groupby(['company_category','province','highLevel_NaceCode'])['staff_cost'].transform('median')
df_melt.loc[df_melt['staff_cost'].isnull(), 'staff_cost'] = median_staff_Cost

In [ ]:
# Checking, no missing values in the Ebitda variable:
df_melt.isnull().sum()

Filling the remainder missing values for staff_cost variable

In [ ]:
# Handle the last missing values staff_cost, reducing the three (DataFrame reference) other level. 
# and instead median Input with the minimum value:
median_staff_Cost = df_melt.groupby(['company_category','province'])['staff_cost'].transform('min')
df_melt.loc[df_melt['staff_cost'].isnull(), 'staff_cost'] = median_staff_Cost

In [ ]:
df_melt.isnull().sum()

### Creating variable Revenue + 1 year

In [ ]:
revenue = df_melt.loc[:, ['nace_code','vat_number','year','revenue']]
revenue.rename(columns={"year":"year_ref", "revenue":"revenue_ref"}, inplace = True)
revenue['year_ref'] = revenue['year_ref'].astype(int)
revenue['year_prev'] = revenue['year_ref'].apply(lambda x: x-1)


In [ ]:
revenue['id'] = revenue.nace_code.str.cat(revenue.vat_number)
revenue['year_prev'] = revenue['year_prev'].astype(str)
revenue['id'] = revenue.id.str.cat(revenue.year_prev)
revenue.drop(['nace_code','vat_number'], axis='columns', inplace=True)
revenue = revenue[revenue.year_prev != '2014']
revenue.shape

### Merging the DataFrames 

In [ ]:
df_final = pd.merge(df_melt, revenue, on = 'id')

In [ ]:
#checking consistency of the 'year' variable
df_final['year'].equals(df_final['year_prev']) 

In [ ]:
df_final.rename(columns={"revenue_ref":"revenue_target", "revenue":"revenue_feature", "size_code":"company_size"}, inplace = True)
df_final = df_final.loc[:,['vat_number','nace_code','highLevel_NaceCode','creation_year','year','province','company_size','total_assets', 'staff_cost', 'revenue_feature', 'revenue_target']]
df_final.head(4)

In [ ]:
#Check: the FEATURES doesn't have any missing values
df_final.isnull().sum()


### Final cleaning data

In [ ]:
# Outliers in the very large companies
# Maintaining only company size: Small, Medium and Large
df_final = df_final[df_final.company_size != 4]

In [ ]:
#discarting companies created in 2019 (without financial history)
df_final = df_final[df_final.creation_year != 2019]

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final.info()

### Creating the final file 'FinancialData.csv'

In [ ]:
# Creating the file 'FinancialData.csv'
columns = ['vat_number', 'nace_code', 'highLevel_NaceCode', 'year', 'province',
       'company_size', 'total_assets', 'staff_cost',
       'revenue_feature', 'revenue_target']
df_final = pd.DataFrame(data=df_final, columns=columns)
df_final.to_csv("FinancialData.csv")